<a href="https://colab.research.google.com/github/EduardoMoraesRitter/NLP-GENERATING-TEXTS/blob/main/Generating_Texts_Recurrent_Neural_Networks_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#dados -

tutorial: https://keras.io/examples/lstm_text_generation/


https://www.neuralnine.com/generating-texts-with-recurrent-neural-networks-in-python/

In [4]:
import random
import numpy as np

import tensorflow as tf

In [ ]:
#shakespeare em ingles
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')

text

In [5]:
filepath = tf.keras.utils.get_file('cpm_musicas.txt',
        'https://raw.githubusercontent.com/EduardoMoraesRitter/NLP-GENERATING-TEXTS/main/cpm_musicas.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')

#text = text[300000:800000]

text

139264/133253 [===============================] - 0s 0us/step


' Tarde de Outubro | CPM 22 - LETRAS \n\n Peguei minhas coisas fui embora\rNão queria mais voltar\rEu nunca quis presenciar o fim\rHá dias que os dias passam devagar\rTudo se foi nada restou pra mimPor isso estou aqui agora\rVou embora sem pensar\rNo que ficou pra começar ali\rMas eu só tinha algumas horas pra voltar\rTudo se foi, nada restou pra mimSão coisas que somente o tempo irá curar\rSe for para nunca mais te ver chorar\rSão coisas que somente o tempo irá curar\rSe foi tudo vai passarPeguei minhas coisas (peguei minhas coisas)\rNão queria mais voltar\rEu nunca quis presenciar o fim\rHá dias que os dias passam devagar\rTudo se foi, nada restou pra mimSão coisas que somente o tempo irá curar\rSe for para nunca mais te ver chorar\rSão coisas que somente o tempo irá curar\rSe foi tudo vai passar\rTudo vai passar\rTudo vai passar \n\n\n\n Irreversível | CPM 22 - LETRAS \n\n A vida me sorri, então\rRecolho os cacos que deixei no chão\rMilhares de recordações transformam tudo em cançõe

In [ ]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

char_to_index
#index_to_char

In [10]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []

In [ ]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])

sentences

In [12]:
x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=np.bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=np.bool)

y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [13]:
for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [ True, False, False, ..., False, False, False]])

#modelo LSTM

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [15]:
model = Sequential()
model.add(LSTM(128,
               input_shape=(SEQ_LENGTH,
                            len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.01))

model.fit(x, y, batch_size=256, epochs=400)

Epoch 1/400
169/169 [==============================] - 1s 9ms/step - loss: 1.5168
Epoch 2/400
169/169 [==============================] - 1s 8ms/step - loss: 1.3538
Epoch 3/400
169/169 [==============================] - 1s 9ms/step - loss: 1.2488
Epoch 4/400
169/169 [==============================] - 1s 8ms/step - loss: 1.1623
Epoch 5/400
169/169 [==============================] - 1s 8ms/step - loss: 1.0914
Epoch 6/400
169/169 [==============================] - 1s 8ms/step - loss: 1.0327
Epoch 7/400
169/169 [==============================] - 1s 9ms/step - loss: 0.9869
Epoch 8/400
169/169 [==============================] - 1s 9ms/step - loss: 0.9426
Epoch 9/400
169/169 [==============================] - 1s 8ms/step - loss: 0.9080
Epoch 10/400
169/169 [==============================] - 1s 8ms/step - loss: 0.8770
Epoch 11/400
169/169 [==============================] - 1s 9ms/step - loss: 0.8526
Epoch 12/400
169/169 [==============================] - 1s 8ms/step - loss: 0.8348
Epoch 13/400


In [18]:
model.save('model_nlp_generating')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_nlp_generating/assets


In [20]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [21]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [24]:
#print(generate_text(300, 0.2))
#print(generate_text(400, 0.4))
#print(generate_text(300, 0.5))
print(generate_text(200, 0.9))
#print(generate_text(300, 0.7))
#print(generate_text(300, 0.8))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


ios passos  Nunca mais vou errar  



 Aria crecurez | CPM 22 - LETRAS 

Thet como fugir o met do melhor rei
